In [3]:
import pandas as pd
import tensorflowjs as tfjs
import keras
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Dropout, Flatten
from keras.utils import to_categorical
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

/Users/philleif/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [26]:
df = pd.read_csv("../data/candles.csv")

In [27]:
df.head()

,mts,open,close,high,low,volume,pair,timeframe,upDownThreeMethods,upsideGapTwoCrows,...,minus_dm,plus_di,minus_di,cvi,adx,apo,rocr,percentChange,strategy,strategyCode
0,1525186800000,0.80930,0.81353,0.82000,0.80800,1.852463e+06,tXRPUSD,1h,0,0,...,0.268851,0.392812,0.261100,0.142994,0.231190,0.542211,1.024081,0.436370,LONG,1
1,1525190400000,0.81501,0.82013,0.82363,0.81088,1.164791e+06,tXRPUSD,1h,0,0,...,0.242450,0.407582,0.233734,0.153669,0.195609,0.561869,1.025419,0.804750,LONG,1
2,1525194000000,0.81908,0.82421,0.82719,0.81712,1.875470e+06,tXRPUSD,1h,0,0,...,0.217935,0.428315,0.212911,0.083404,0.167776,0.585394,1.033751,0.495019,LONG,1
3,1525197600000,0.82454,0.82680,0.82910,0.81970,1.178272e+06,tXRPUSD,1h,0,0,...,0.195171,0.427347,0.193917,0.052363,0.146119,0.609298,1.027017,0.313256,LONG,1
4,1525201200000,0.82670,0.82536,0.82886,0.82063,7.016413e+05,tXRPUSD,1h,0,0,...,0.174033,0.402241,0.177555,0.049270,0.126010,0.626648,1.012923,-0.174469,SHORT,0


In [32]:
def create_model(optimizer="adam", init_mode="uniform", activation="sigmoid"):
    model = Sequential()

    model.add(Dense(48, kernel_initializer=init_mode, input_shape=(46,), activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(24, kernel_initializer=init_mode, activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(12, kernel_initializer=init_mode, activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer=init_mode, activation="sigmoid"))

    model.compile(loss="binary_crossentropy",
                  optimizer=optimizer,
                  metrics=["accuracy"])
    return model

In [29]:
X = df[["adx", "cci", "rocr", "hikkake", "harami", "closingMarubozu", 
        "onNeck", "longLineCandle", "hikkakeModified", "apo", "cmo", "adx",
        "cvi", "plus_dm", "minus_dm", "plus_di", "minus_di", "dpo", "kvo",
        "fosc", "fisher", "fisher_signal", "dx", "linregslope", "macd", "macd_signal", 
        "macd_histogram", "mfi", "mom", "obv", "ppo", "pvi", "rsi", "stoch_k", "stoch_d",
        "atr", "trix", "ultosc", "vosc", "willr", "cmo", "cci", "adosc", "adxr", "ao",
        "aroonosc"
       ]].values

y = df["strategyCode"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [53]:
model = KerasClassifier(build_fn=create_model, verbose=0)

batch_size = [128]
epochs = [500]
optimizer = ['Adam']
init_mode = ['uniform']
activation = ['softsign']

param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer, init_mode=init_mode, activation=activation)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

In [54]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.829147 using {'activation': 'softsign', 'batch_size': 128, 'epochs': 500, 'init_mode': 'uniform', 'optimizer': 'Adam'}
0.829147 (0.002025) with: {'activation': 'softsign', 'batch_size': 128, 'epochs': 500, 'init_mode': 'uniform', 'optimizer': 'Adam'}
0.825206 (0.005427) with: {'activation': 'softsign', 'batch_size': 128, 'epochs': 500, 'init_mode': 'lecun_uniform', 'optimizer': 'Adam'}
0.825994 (0.004582) with: {'activation': 'softsign', 'batch_size': 128, 'epochs': 500, 'init_mode': 'normal', 'optimizer': 'Adam'}
0.515711 (0.007595) with: {'activation': 'softsign', 'batch_size': 128, 'epochs': 500, 'init_mode': 'zero', 'optimizer': 'Adam'}
0.825881 (0.011508) with: {'activation': 'softsign', 'batch_size': 128, 'epochs': 500, 'init_mode': 'glorot_normal', 'optimizer': 'Adam'}
0.823404 (0.008115) with: {'activation': 'softsign', 'batch_size': 128, 'epochs': 500, 'init_mode': 'glorot_uniform', 'optimizer': 'Adam'}
0.823967 (0.005414) with: {'activation': 'softsign', 'batch_size':

In [37]:
optimal_model = create_model(optimizer="Adam", init_mode="uniform", activation="softsign")

#y_labels = to_categorical(y_train, num_classes=2)

optimal_model.fit(X_train, y_train, epochs=250, batch_size=256, verbose=1)

Epoch 1/250
35003/35003 [==============================] - 1s 20us/step - loss: 0.6063 - acc: 0.6911
Epoch 2/250
35003/35003 [==============================] - 0s 7us/step - loss: 0.4835 - acc: 0.7808
Epoch 3/250
35003/35003 [==============================] - 0s 7us/step - loss: 0.4572 - acc: 0.7899
Epoch 4/250
35003/35003 [==============================] - 0s 7us/step - loss: 0.4459 - acc: 0.7951
Epoch 5/250
35003/35003 [==============================] - 0s 7us/step - loss: 0.4344 - acc: 0.7989
Epoch 6/250
35003/35003 [==============================] - 0s 7us/step - loss: 0.4325 - acc: 0.7994
Epoch 7/250
35003/35003 [==============================] - 0s 7us/step - loss: 0.4275 - acc: 0.8001
Epoch 8/250
35003/35003 [==============================] - 0s 7us/step - loss: 0.4216 - acc: 0.8042
Epoch 9/250
35003/35003 [==============================] - 0s 7us/step - loss: 0.4248 - acc: 0.8025
Epoch 10/250
35003/35003 [==============================] - 0s 7us/step - loss: 0.4174 - acc: 0.806

In [38]:
#y_test_labels = to_categorical(y_test, num_classes=2)
evaluation = optimal_model.evaluate(x=X_test, y=y_test, steps=100)
optimal_model.metrics_names

100/100 [==============================] - 1s 5ms/step


['loss', 'acc']

In [39]:
print(evaluation)

[0.35523146390914917, 0.8363204002380371]


In [40]:
tfjs.converters.save_keras_model(optimal_model, "../public/model/")